In [7]:
#Importing Libraries 
import pandas as pd
import statistics

In [2]:
df = pd.read_csv("Final_CleanedData.csv") #Reading in Dataframe 
data = df[['id' , 'domain_id', 'batch_id' , 'ean_13', 'seller_name' ,  'found' , 'type' , 'sold_by_third_party' , 
           'fulfilled_by_third_party', 'has_buy_button' , 'started_at' , 'primary_sale_type']] #Creating a subset of data with specific fields 
data.head()

/Users/AmandaFakhoury/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (29,46,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,domain_id,batch_id,ean_13,seller_name,found,type,sold_by_third_party,fulfilled_by_third_party,has_buy_button,started_at,primary_sale_type
0,124273619,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781402738159,SuperBookDeals---,t,Hardcover,t,t,t,18/11/2019 20:12:05.936306,For Sale
1,124280803,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781454919735,SuperBookDeals---,t,Board book,t,t,t,18/11/2019 20:12:05.936306,For Sale
2,124281694,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781454922377,SuperBookDeals---,t,Hardcover,t,t,t,18/11/2019 20:12:05.936306,For Sale
3,124284134,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781454929284,SuperBookDeals---,t,Board book,t,t,t,18/11/2019 20:12:05.936306,For Sale
4,124284363,a7ed8222-280f-4030-aab9-a76000f621d6,1185,9781454929666,SuperBookDeals---,t,Board book,t,t,t,18/11/2019 20:12:05.936306,For Sale


In [3]:
#Date Column
data['started_at'] = pd.to_datetime(data['started_at'] , dayfirst = True) #Converting column to datetime form 
data['Year'] = data['started_at'].dt.year #Creating year column
data['Month'] = data['started_at'].dt.month #Creating Month column
data['Day'] = data['started_at'].dt.day #Creating Day Column

/Users/AmandaFakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/AmandaFakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/AmandaFakhoury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [4]:
#Creating a dataframe for each month
Months = list(data['Month'].unique()) #Creating a list of the months 
Months_df = [] 
for month in Months: 
    Months_df.append(data.loc[data['Month'] == month])  

In [9]:
stats_data_dfs = [] 
for mdf in Months_df: 
    
    Days = list(mdf['Day'].unique()) #Create a list of the days 
    ld = len(Days) #Length of Days 
    
    
    saletype = list(mdf['primary_sale_type'].unique()) #Creating a list of sale types
    
    #Creating a dataframe for each primary sale type
    saletype_dfs = [] 
    for t in saletype: 
        saletype_dfs.append(mdf.loc[mdf['primary_sale_type'] == t])
        
    
    new_data_dfs = [] 
    for d1 in saletype_dfs: #Looping through dataframe by sale type
        
        sellers = list(d1['seller_name'].unique()) #creating a list of seller names 
        
        
        numb_eans = [] 
        avg_buywins = [] 
        avg_percent = []
        for s in sellers: 
            buycount = [] 
            buypct = []
            d = d1.loc[d1['seller_name'] == s] #Creating a dataframe for each seller 
            numb_eans.append(len(d['ean_13'].unique())) #Getting number of isbn 

            eans = list(d['ean_13'].unique()) #Creating a list of isbns
            for e in eans: 
                dd = d.loc[d['ean_13'] == e] 
                buycount.append(dd['has_buy_button'].count()) #Getting number of buy button wins for each isbn 

            for b in buycount: 
                buypct.append((b/ld) * 100) #Getting percentage of buy button wins 



            avg_buywins.append(statistics.mean(buycount)) #Getting an avg count of buy button wins for each seller
            avg_percent.append(statistics.mean(buypct)) #Getting an avg Percentage of buy button wins for each seller 
            
        #Creating dataframe with final numbers  
        df_sellers = pd.DataFrame({'seller_name': sellers, 
                          'Number of Titles': numb_eans, 
                          'Avg Buy Wins': avg_buywins, 
                          'Avg Buy Wins Percentage': avg_percent}) 
        
        new_data_dfs.append(df_sellers)
    
    for tp , d2 in zip(saletype, new_data_dfs):
        d2['Primary_Sale_Type'] = tp
        
    mdata = pd.concat(new_data_dfs)
    
    stats_data_dfs.append(mdata)

In [13]:
for m , d4 in zip(Months,  stats_data_dfs): 
    d4['Month'] = m

In [15]:
finaldata = pd.concat(stats_data_dfs) #Merging Data together 
finaldata.head()

,seller_name,Number of Titles,Avg Buy Wins,Avg Buy Wins Percentage,Primary_Sale_Type,Month
0,SuperBookDeals---,739.0,10.0,36.296797,For Sale,11
1,indoobestsellers,346.0,12.0,41.868979,For Sale,11
2,Book Depository US,1359.0,9.0,30.596026,For Sale,11
3,FOX_BOOKS,11.0,7.0,26.363636,For Sale,11
4,ChronosBooks,1.0,24.0,80.000000,For Sale,11


In [17]:
typesale = list(finaldata['Primary_Sale_Type'].unique()) #Creating a list of sale type 
dfs_saletype = [] 
for t in typesale:
    dfs_saletype.append(finaldata.loc[finaldata['Primary_Sale_Type'] == t]) #Creating a dataframe for each sale type

In [18]:
n_datadfs = [] 
for d in dfs_saletype: 
    
    sellers1 = list(d['seller_name'].unique())
    
    df2_sellers = []
    for s in sellers1: 

        d = d.loc[d['seller_name'] == s]
        d1 = d[['seller_name', 'Number of Titles', 'Avg Buy Wins' , 'Avg Buy Wins Percentage' ]]
        months = list(d['Month'])
        saletype = list(d['Primary_Sale_Type'])
        perc = list(d['Avg Buy Wins Percentage'])
        title = list(d['Number of Titles'])
        wins = list(d['Avg Buy Wins'])

        diff1 = d1.set_index('seller_name').diff()

        diff2 = diff1.reset_index()
        diff2['Month'] = months
        diff2['Primary_Sale_Type'] = saletype
        diff2['AvgWinsPercent'] = perc
        diff2['AvgWins'] = wins
        diff2['NumberTitles'] = title

        df2_sellers.append(diff2)
        
        
    dfinal = pd.concat(df2_sellers)
    n_datadfs.append(dfinal)

In [26]:
final_data2 = pd.concat(n_datadfs)
final_data2.head()

,seller_name,Number of Titles,Avg Buy Wins,Avg Buy Wins Percentage,Month,Primary_Sale_Type,AvgWinsPercent,AvgWins,NumberTitles
0,SuperBookDeals---,NaN,NaN,NaN,11.0,For Sale,36.296797,10.0,739.0
1,SuperBookDeals---,-51.0,-3.0,-11.539046,12.0,For Sale,24.757752,7.0,688.0
2,SuperBookDeals---,524.0,-2.0,-5.533329,4.0,For Sale,19.224422,5.0,1212.0
3,SuperBookDeals---,-1206.0,-4.0,-14.385713,5.0,For Sale,4.838710,1.0,6.0
0,SuperBookDeals---,NaN,NaN,NaN,11.0,Rental,38.244576,11.0,169.0


In [27]:
final_data2.columns = ['seller_name', 'Change in Number of Titles' , 'Change in Avg Buy Wins', 'Change in Avg Buy Wins Percentage', 'Month', 'Primary_Sale_Type', 'AvgWinsPercent', 'AvgWins', 'NumberTitles']




In [28]:
finalData3 = final_data2.fillna(value=0)


category = [] 
for i in finalData3['Change in Avg Buy Wins Percentage']: 
    if i < 0 : 
        category.append("Percentage Descrease")
    elif i == 0: 
        category.append("No Change")
    else: 
        category.append("Percenatge Increase")
        
        
finalData3['Change Percentage Category'] = category
finalData3.head()

,seller_name,Change in Number of Titles,Change in Avg Buy Wins,Change in Avg Buy Wins Percentage,Month,Primary_Sale_Type,AvgWinsPercent,AvgWins,NumberTitles,Change Percentage Category
0,SuperBookDeals---,0.0,0.0,0.000000,11.0,For Sale,36.296797,10.0,739.0,No Change
1,SuperBookDeals---,-51.0,-3.0,-11.539046,12.0,For Sale,24.757752,7.0,688.0,Percentage Descrease
2,SuperBookDeals---,524.0,-2.0,-5.533329,4.0,For Sale,19.224422,5.0,1212.0,Percentage Descrease
3,SuperBookDeals---,-1206.0,-4.0,-14.385713,5.0,For Sale,4.838710,1.0,6.0,Percentage Descrease
0,SuperBookDeals---,0.0,0.0,0.000000,11.0,Rental,38.244576,11.0,169.0,No Change


In [ ]:
finalData3.to_csv('ThirdPartySellers_Difference.csv' , index = False) #Save to csv 
finalData3.to_excel("Sellers_Difference.xlsx") #Save to excel 